# Supplementary Behavioral Results

This script loads the trial-level behavioral data and individual-level neural data and runs mixed effects models to test the effect of node type and network type on RT as well as the moderating effect of brain connectivity. We separately include mean brain activation and head motion as covariates to control for these confounds and test whether connectivity moderates network learning above and beyond the effect of brain activity or head motion. Finally, we also test whether functional connectivity in a priori hubs identified in the main manuscript are correlated with social traits or IQ." 

## Step 1: Set up

In [1]:
## Load packages
library(reshape2)
library(plyr)
library(QuantPsyc)
library(psych)
library(ggplot2)
library(effects)
library(lme4)
library(lmerTest)

## Set Working Directory to be location where CSV files are stored (change to the location you downloaded data to)
wd1<-"/Users/steventompson/Git/tompson_netlearn_fmri/data/subj_data/"
wd2<-"/Users/steventompson/Git/tompson_netlearn_fmri/data/ppi_zscores/"
wd3<-"/Users/steventompson/Git/tompson_netlearn_fmri/data/glm_means/"

Warning message:
“package ‘reshape2’ was built under R version 3.4.3”Loading required package: boot
Warning message:
“package ‘boot’ was built under R version 3.4.1”Loading required package: MASS
Warning message:
“package ‘MASS’ was built under R version 3.4.4”
Attaching package: ‘QuantPsyc’

The following object is masked from ‘package:base’:

    norm


Attaching package: ‘psych’

The following object is masked from ‘package:boot’:

    logit

Warning message:
“package ‘ggplot2’ was built under R version 3.4.4”
Attaching package: ‘ggplot2’

The following objects are masked from ‘package:psych’:

    %+%, alpha

Warning message:
“package ‘effects’ was built under R version 3.4.1”Loading required package: carData
Warning message:
“package ‘carData’ was built under R version 3.4.4”lattice theme set by effectsTheme()
See ?effectsTheme for details.
Warning message:
“package ‘lme4’ was built under R version 3.4.4”Loading required package: Matrix
Warning message:
“package ‘Matrix’ was built

In [2]:
sessionInfo()

R version 3.4.0 (2017-04-21)
Platform: x86_64-apple-darwin15.6.0 (64-bit)
Running under: macOS  10.14.6

Matrix products: default
BLAS: /Library/Frameworks/R.framework/Versions/3.4/Resources/lib/libRblas.0.dylib
LAPACK: /Library/Frameworks/R.framework/Versions/3.4/Resources/lib/libRlapack.dylib

locale:
[1] en_US.UTF-8/en_US.UTF-8/en_US.UTF-8/C/en_US.UTF-8/en_US.UTF-8

attached base packages:
[1] stats     graphics  grDevices utils     datasets  methods   base     

other attached packages:
 [1] lmerTest_3.0-1 lme4_1.1-19    Matrix_1.2-15  effects_4.0-0  carData_3.0-2 
 [6] ggplot2_3.1.0  psych_1.8.12   QuantPsyc_1.5  MASS_7.3-51.1  boot_1.3-20   
[11] plyr_1.8.4     reshape2_1.4.3

loaded via a namespace (and not attached):
 [1] pbdZMQ_0.3-3      tidyselect_0.2.5  repr_0.19.1       purrr_0.2.5      
 [5] splines_3.4.0     lattice_0.20-38   colorspace_1.4-0  htmltools_0.3.6  
 [9] base64enc_0.1-3   survival_2.43-3   rlang_0.3.1       pillar_1.3.1     
[13] nloptr_1.2.1      foreign_0.8

In [3]:
## Load trial-level behavioral data
tdf<-read.csv(paste(wd1,"tompson_netlearn_fmri_trial_data.csv",sep=""))

tdfNS<-tdf[tdf$Cond=="NS",]
tdfSoc<-tdf[tdf$Cond=="Soc",]
head(tdf) 

X,path,pID,walk,trialNum,altered,correct_raw,onset_raw,resp_raw,rt_raw,order,Cond,pre,transition,grouping
1,img/18_circ.png,1,2,0,False,0,0.00288415,NA,NA,0,NS,0,0,x
2,img/4_circ.png,1,1,1,False,0,1.51719999,4,977.3271,1,NS,0,0,x
3,img/13_circ.png,1,0,2,False,1,3.01679420,1,478.4188,2,NS,0,0,x
4,img/18_circ_rotated.png,1,2,3,True,0,4.51468945,1,681.2568,3,NS,0,0,x
5,img/15_circ.png,1,3,4,False,0,6.01897907,4,327.5101,4,NS,0,0,x
6,img/18_circ.png,1,2,5,False,1,7.51567698,1,263.9382,5,NS,0,0,x


## Step 2: Exclude bad subjects and trials

In [4]:
#################################################
### check summary data for each subject to determine who to exclude
#################################################

print('')
print('Combined Accuracy')
print(tapply(tdf$correct,tdf$pID,function(i){sum(i,na.rm=T)/2000})) #percent correct trials

print('')
print('Non-Social Accuracy')
print(tapply(tdfNS$correct,tdfNS$pID,function(i){sum(i,na.rm=T)/1000})) #percent correct trials

print('')
print('Social Accuracy')
print(tapply(tdfSoc$correct,tdfSoc$pID,function(i){sum(i,na.rm=T)/1000})) #percent correct trials

[1] ""
[1] "Combined Accuracy"
     1      2      4      5      8      9     11     12     13     15     16 
0.7540 0.9230 0.8975 0.9010 0.9005 0.9085 0.8535 0.8845 0.3640 0.6020 0.9025 
    18     19     20     21     23     24     25     26     27     28     29 
0.8705 0.8610 0.4290 0.9580 0.7220 0.8960 0.9280 0.7190 0.8875 0.2390 0.9295 
    30     31     32     33     34     35     36     37     39 
0.8950 0.8590 0.7935 0.9025 0.7565 0.8320 0.8385 0.6340 0.9120 
[1] ""
[1] "Non-Social Accuracy"
    1     2     4     5     8     9    11    12    13    15    16    18    19 
0.753 0.916 0.887 0.926 0.913 0.873 0.886 0.884 0.728 0.645 0.892 0.861 0.881 
   20    21    23    24    25    26    27    28    29    30    31    32    33 
0.345 0.958 0.748 0.892 0.925 0.760 0.869 0.286 0.939 0.896 0.874 0.823 0.923 
   34    35    36    37    39 
0.636 0.818 0.801 0.636 0.910 
[1] ""
[1] "Social Accuracy"
    1     2     4     5     8     9    11    12    15    16    18    19    20 
0.755 0.93

In [5]:
#################################################
### Exclude subjects with fewer than 70% correct responses

#NOTE: 18 subjects had greater than 70% correct
#NOTE: 2 subjects missed greater than 50% of trials (20=43%, 28=24%)
#NOTE: 2 subjects missed greater than 30% of trials (15=60%, 37=64%) but will keep them in for now
#NOTE: 1 subject had to abort because they were feeling claustrophobic (13)
#NOTE: 1 subject had to be excluded due to brain artifact (4=missing back part of parietal cortex)
#################################################

tdf<-subset(tdf,pID!= 1)
tdf<-subset(tdf,pID!= 4)
tdf<-subset(tdf,pID!= 13)
#tdf<-subset(tdf,pID!= 15)
tdf<-subset(tdf,pID!= 20)
tdf<-subset(tdf,pID!= 28)
#tdf<-subset(tdf,pID!= 37)

In [6]:
#################################################
### Exclude bad trials and trials we won't use in analyses
#################################################

#remove incorrect trials
tdf2<-subset(tdf,tdf$correct_raw==1)

#remove altered trials
tdf2<-subset(tdf2,tdf2$altered=="False")


#Get rid of impossible reaction times (i.e., less than 100, >1500)
tdf2 <- subset(tdf2, rt_raw < 1500)
tdf2 <- subset(tdf2, rt_raw > 100)

##remove Rts > 3 Sd from the mean
tdf2 = ddply(tdf2, .(pID), transform, sRT = scale(rt_raw))
tdf2= subset(tdf2, abs(sRT) < 3)

#check # of transition trials left after cleaning
print('# of transition trials')
print(tapply(tdf2$grouping,tdf2$pID,function(i){sum(i=="transition",na.rm=T)})) #number of transition trials

#convert trial grouping variable to a factor
tdf2$grouping <- factor(tdf2$grouping, levels = c("x", "transition"))


rm(tdf,tdfNS,tdfSoc)

print('')
print(paste('Total # of subjects:',length(unique(tdf2$pID))))

[1] "# of transition trials"
  2   5   8   9  11  12  15  16  18  19  21  23  24  25  26  27  29  30  31  32 
 97  99 108 103  97  99  65  91  89  83 104  97  94  99  99  98  97 107  86  82 
 33  34  35  36  37  39 
 85  93  90  89  67  95 
[1] ""
[1] "Total # of subjects: 26"


## Step 3: Load and merge neural data

In [7]:
#################################################
### Load data
#################################################

#Load IDs to link behavioral and neural data
subjIDs<-read.csv(paste(wd1,"netLearn_IDs_26subjs.csv",sep="/"))

#Load head motion data file
hmData<-read.csv(paste(wd1,"netLearn_headMotion_26subjs.csv",sep="/"))

#Define function for creating df
create_df<-function(df_folder,dname1,dname2,df1,df2,add_cov=FALSE){
  filepath<-paste(df_folder,"netLearn_",dname1,
                  "_26subjs_",dname2,".csv",sep="")
  new.df<-read.csv(filepath,header=T,as.is=T)
  new.df<-cbind(df1,new.df)
  if(add_cov==TRUE){
      new.df<-merge(new.df,df2,by="pID")
  }
    
  new.df$task<-gsub("Data","",unlist(strsplit(dname1,"_"))[1])
  return(new.df)
}

# Load neural data and merge with behavioral/head motion data
nsData_gc<-create_df(wd2,"NSData_zscores","sighubs_global_conn",subjIDs,hmData,add_cov=TRUE)
socData_gc<-create_df(wd2,"SocData_zscores","sighubs_global_conn",subjIDs,hmData,add_cov=TRUE)

nsData_ss<-create_df(wd2,"NSData_zscores","sighubs_to_sighubs",subjIDs,hmData)
socData_ss<-create_df(wd2,"SocData_zscores","sighubs_to_sighubs",subjIDs,hmData)

nsData_glm<-create_df(wd3,"NSData_glm_transition","sighubs_activation",subjIDs,hmData)
socData_glm<-create_df(wd3,"SocData_glm_transition","sighubs_activation",subjIDs,hmData)


#################################################
# Combine neural dataframes
#################################################

longData_gc<-rbind(socData_gc,nsData_gc)
longData_ss<-rbind(socData_ss,nsData_ss)
longData_glm<-rbind(socData_glm,nsData_glm)


longData<-merge(longData_gc,longData_ss,by = c("pID","task","CondNum","rowID","scanID"))
longData<-merge(longData,longData_glm,by = c("pID","task","CondNum","rowID","scanID"))

## Step 4: Test correlations between connectivity measures and brain activity

In [8]:
hublist1<-c("combhubs_global", "Hippocampus_L_global","Hippocampus_R_global",
            "diffhubs_global","TPJ_L_global","TPJ_R_global",
            "diffhubs_ns_global","dmPFC_L_global","dmPFC_R_global","lPFC_L_global","lPFC_R_global")
hublist2<-c("ws_combhubs","ws_nshubs","ws_sochubs")
glmlist1<-c("combhubs_glm", "Hippocampus_L_glm","Hippocampus_R_glm",
            "diffhubs_glm","TPJ_L_glm","TPJ_R_glm",
            "diffhubs_ns_glm","dmPFC_L_glm","dmPFC_R_glm","lPFC_L_glm","lPFC_R_glm")
glmlist2<-c("combhubs_glm", "diffhubs_ns_glm","diffhubs_glm")

In [9]:
for(i in 1:length(hublist1)){
    name1<-hublist1[i]
    name2<-glmlist1[i]
    test<-cor.test(longData[,name1],longData[,name2])
    print(paste(name1,'corr with',name2))
    print(paste('r =',round(test$estimate,3),'p =',round(test$p.value,3)))
    print('')
}

[1] "combhubs_global corr with combhubs_glm"
[1] "r = 0.09 p = 0.525"
[1] ""
[1] "Hippocampus_L_global corr with Hippocampus_L_glm"
[1] "r = 0.057 p = 0.686"
[1] ""
[1] "Hippocampus_R_global corr with Hippocampus_R_glm"
[1] "r = -0.02 p = 0.889"
[1] ""
[1] "diffhubs_global corr with diffhubs_glm"
[1] "r = 0.107 p = 0.452"
[1] ""
[1] "TPJ_L_global corr with TPJ_L_glm"
[1] "r = 0.129 p = 0.361"
[1] ""
[1] "TPJ_R_global corr with TPJ_R_glm"
[1] "r = 0.064 p = 0.652"
[1] ""
[1] "diffhubs_ns_global corr with diffhubs_ns_glm"
[1] "r = -0.002 p = 0.989"
[1] ""
[1] "dmPFC_L_global corr with dmPFC_L_glm"
[1] "r = -0.054 p = 0.702"
[1] ""
[1] "dmPFC_R_global corr with dmPFC_R_glm"
[1] "r = 0.069 p = 0.627"
[1] ""
[1] "lPFC_L_global corr with lPFC_L_glm"
[1] "r = -0.018 p = 0.902"
[1] ""
[1] "lPFC_R_global corr with lPFC_R_glm"
[1] "r = 0.065 p = 0.646"
[1] ""


In [10]:
for(i in 1:length(hublist2)){
    name1<-hublist2[i]
    name2<-glmlist2[i]
    test<-cor.test(longData[,name1],longData[,name2])
    print(paste(name1,'corr with',name2))
    print(paste('r =',round(test$estimate,3),'p =',round(test$p.value,3)))
    print('')
}

[1] "ws_combhubs corr with combhubs_glm"
[1] "r = 0.126 p = 0.375"
[1] ""
[1] "ws_nshubs corr with diffhubs_ns_glm"
[1] "r = -0.231 p = 0.099"
[1] ""
[1] "ws_sochubs corr with diffhubs_glm"
[1] "r = 0.119 p = 0.399"
[1] ""


## Step 5: Combine neural and trial-level data

In [11]:
#################################################
# Merge neural and trial-level data
#################################################

combData2<-merge(tdf2,longData,by.x = c("pID","Cond"),by.y=c("pID","task"))

head(combData2)

Warning message in merge.data.frame(tdf2, longData, by.x = c("pID", "Cond"), by.y = c("pID", :
“column names ‘X.x’, ‘X.y’ are duplicated in the result”

pID,Cond,X.x,path,walk,trialNum,altered,correct_raw,onset_raw,resp_raw,⋯,Hippocampus_L_glm,Hippocampus_R_glm,diffhubs_glm,TPJ_L_glm,TPJ_R_glm,diffhubs_ns_glm,dmPFC_L_glm,dmPFC_R_glm,lPFC_L_glm,lPFC_R_glm
11,NS,2002,img/5_circ.png,9,1,False,1,1.513404,1,⋯,0.3777449,0.09631367,1.111413,0.2089689,1.562636,0.6278472,-0.130339,0.7294553,-0.5032621,1.340994
11,NS,2003,img/1_circ.png,8,2,False,1,3.016448,1,⋯,0.3777449,0.09631367,1.111413,0.2089689,1.562636,0.6278472,-0.130339,0.7294553,-0.5032621,1.340994
11,NS,2005,img/1_circ.png,8,4,False,1,6.018421,1,⋯,0.3777449,0.09631367,1.111413,0.2089689,1.562636,0.6278472,-0.130339,0.7294553,-0.5032621,1.340994
11,NS,2007,img/3_circ.png,4,6,False,1,9.018356,1,⋯,0.3777449,0.09631367,1.111413,0.2089689,1.562636,0.6278472,-0.130339,0.7294553,-0.5032621,1.340994
11,NS,2009,img/17_circ.png,1,8,False,1,12.018660,1,⋯,0.3777449,0.09631367,1.111413,0.2089689,1.562636,0.6278472,-0.130339,0.7294553,-0.5032621,1.340994
11,NS,2010,img/18_circ.png,3,9,False,1,13.518142,1,⋯,0.3777449,0.09631367,1.111413,0.2089689,1.562636,0.6278472,-0.130339,0.7294553,-0.5032621,1.340994


In [12]:
#################################################
# Convert key variables to z-scores
#################################################

create_zscores<-function(varlist,df1){
  newlist<-c()
  for(var in varlist){
    newvar<-paste(var,"z",sep="_")
    newlist<-c(newlist,newvar)
    df1[,newvar]<-as.numeric(Make.Z(df1[,var]))
  }
  return(list(newlist,df1))
}

brain_var_list<-unique(c(hublist1,hublist2,glmlist1,glmlist2))
zscores<-create_zscores(brain_var_list,combData2)
zvars<-zscores[[1]]
combData2<-zscores[[2]]

## Step 6: Brain Connectivity Moderation Analyses with Confounds

In [13]:
sighubs1<-c("Hippocampus_L_global_z","diffhubs_global_z","TPJ_L_global_z","dmPFC_L_global_z")
sighubs2<-c("ws_sochubs_z")
sighubs<-c(sighubs1,sighubs2)

i<-match("diffhubs_global",hublist1)

In [14]:
#################################################
### Clean trial df
#################################################

# Create function to clean df
clean_trial_df<-function(df1){
    # Add order info to trial-level df
    df1$order<-NA
    for(subj in unique(df1$pID)){
        df1$order[df1$pID==subj]<-subjIDs$CondNum[subjIDs$pID==subj]
    }

    #z-score variables
    df1$strial<-as.numeric(Make.Z(df1$trialNum))

    #contrast coding
    contrasts(df1$grouping) <- contr.helmert(2) 
    contrasts(df1$grouping)
  
    df1$Cond<-factor(df1$Cond)
    contrasts(df1$Cond) <- contr.helmert(2) 
    contrasts(df1$Cond)
  
    df1$order<-factor(df1$order)
    contrasts(df1$order) <- contr.helmert(2) 
    contrasts(df1$order)
    
    #log-transform variables
    df1$rt_log<-log(df1$rt_raw)
    df1$rt_log<-df1$rt_log-mean(df1$rt_log)
    return(df1)
}

#################################################
### Clean merged trial/neural data
#################################################
combData2<-clean_trial_df(combData2)


nsData2<-combData2[combData2$Cond=='NS',]
socData2<-combData2[combData2$Cond=='Soc',]

In [15]:
mm_TPJ<-lmer(rt_log~grouping*Cond*order*strial*TPJ_L_global_z + (1+grouping+Cond+strial | pID),data=combData2)
mm_TPJ_glm<-lmer(rt_log~grouping*Cond*order*strial*TPJ_L_global_z + diffhubs_glm + (1+grouping+Cond+strial | pID),data=combData2)
mm_TPJ_hm<-lmer(rt_log~grouping*Cond*order*strial*TPJ_L_global_z + mean_fd_all + (1+grouping+Cond+strial | pID),data=combData2)

mm_sochubs_global<-lmer(rt_log~grouping*Cond*order*strial*diffhubs_global_z + (1+grouping+Cond+strial | pID),data=combData2)
mm_sochubs_global_glm<-lmer(rt_log~grouping*Cond*order*strial*diffhubs_global_z + diffhubs_glm + (1+grouping+Cond+strial | pID),data=combData2)
mm_sochubs_global_hm<-lmer(rt_log~grouping*Cond*order*strial*diffhubs_global_z + mean_fd_all + (1+grouping+Cond+strial | pID),data=combData2)

mm_hipp<-lmer(rt_log~grouping*Cond*order*strial*Hippocampus_L_global_z + (1+grouping+Cond+strial | pID),data=combData2)
mm_hipp_glm<-lmer(rt_log~grouping*Cond*order*strial*Hippocampus_L_global_z + diffhubs_glm + (1+grouping+Cond+strial | pID),data=combData2)
mm_hipp_hm<-lmer(rt_log~grouping*Cond*order*strial*Hippocampus_L_global_z + mean_fd_all + (1+grouping+Cond+strial | pID),data=combData2)

mm_dmPFC<-lmer(rt_log~grouping*Cond*order*strial*dmPFC_L_global_z + (1+grouping+Cond+strial | pID),data=combData2)
mm_dmPFC_glm<-lmer(rt_log~grouping*Cond*order*strial*dmPFC_L_global_z + diffhubs_glm + (1+grouping+Cond+strial | pID),data=combData2)
mm_dmPFC_hm<-lmer(rt_log~grouping*Cond*order*strial*dmPFC_L_global_z + mean_fd_all + (1+grouping+Cond+strial | pID),data=combData2)

mm_ws_sochubs<-lmer(rt_log~grouping*Cond*order*strial*ws_sochubs_z + (1+grouping+Cond+strial | pID),data=combData2)
mm_ws_sochubs_glm<-lmer(rt_log~grouping*Cond*order*strial*ws_sochubs_z + diffhubs_glm + (1+grouping+Cond+strial | pID),data=combData2)
mm_ws_sochubs_hm<-lmer(rt_log~grouping*Cond*order*strial*ws_sochubs_z + mean_fd_all + (1+grouping+Cond+strial | pID),data=combData2)


In [16]:
mm_og_3way_pvals<-c(summary(mm_hipp)$coefficients[21,][['Pr(>|t|)']],
               summary(mm_ws_sochubs)$coefficients[21,][['Pr(>|t|)']],
               summary(mm_dmPFC)$coefficients[21,][['Pr(>|t|)']],
               summary(mm_sochubs_global)$coefficients[21,][['Pr(>|t|)']],
               summary(mm_TPJ)$coefficients[21,][['Pr(>|t|)']])


mm_glm_3way_pvals<-c(summary(mm_hipp_glm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_ws_sochubs_glm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_dmPFC_glm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_sochubs_global_glm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_TPJ_glm)$coefficients[22,][['Pr(>|t|)']])

mm_hm_3way_pvals<-c(summary(mm_hipp_hm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_ws_sochubs_hm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_dmPFC_hm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_sochubs_global_hm)$coefficients[22,][['Pr(>|t|)']],
               summary(mm_TPJ_hm)$coefficients[22,][['Pr(>|t|)']])

mm_og_2way_pvals<-c(summary(mm_hipp)$coefficients[13,][['Pr(>|t|)']],
               summary(mm_ws_sochubs)$coefficients[13,][['Pr(>|t|)']],
               summary(mm_dmPFC)$coefficients[13,][['Pr(>|t|)']],
               summary(mm_sochubs_global)$coefficients[13,][['Pr(>|t|)']],
               summary(mm_TPJ)$coefficients[13,][['Pr(>|t|)']])

mm_glm_2way_pvals<-c(summary(mm_hipp_glm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_ws_sochubs_glm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_dmPFC_glm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_sochubs_global_glm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_TPJ_glm)$coefficients[14,][['Pr(>|t|)']])

mm_hm_2way_pvals<-c(summary(mm_hipp_hm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_ws_sochubs_hm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_dmPFC_hm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_sochubs_global_hm)$coefficients[14,][['Pr(>|t|)']],
               summary(mm_TPJ_hm)$coefficients[14,][['Pr(>|t|)']])


### Print results for mixed models controlling for confounds

In [17]:
print('Results with Brain Activity as Covariate')
round(summary(mm_hipp_glm)$coefficients[c(14,22),],3)
round(summary(mm_ws_sochubs_glm)$coefficients[c(14,22),],3)
round(summary(mm_dmPFC_glm)$coefficients[c(14,22),],3)
round(summary(mm_sochubs_global_glm)$coefficients[c(14,22),],3)
round(summary(mm_TPJ_glm)$coefficients[c(14,22),],3)

print('FDR-corrected p-values')
print(round(p.adjust(mm_glm_3way_pvals,method="fdr"),3))
print(round(p.adjust(mm_glm_2way_pvals,method="fdr"),3))

[1] "Results with Brain Activity as Covariate"


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:Hippocampus_L_global_z,0.001,0.003,139.986,0.508,0.612
grouping1:Cond1:Hippocampus_L_global_z,0.007,0.003,231.577,2.552,0.011


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:ws_sochubs_z,0.007,0.003,153.729,2.262,0.025
grouping1:Cond1:ws_sochubs_z,0.008,0.003,209.359,2.921,0.004


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:dmPFC_L_global_z,0.001,0.003,198.412,0.459,0.646
grouping1:Cond1:dmPFC_L_global_z,-0.008,0.003,192.413,-2.876,0.004


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:diffhubs_global_z,0.008,0.003,190.633,2.773,0.006
grouping1:Cond1:diffhubs_global_z,0.002,0.003,188.359,0.665,0.507


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:TPJ_L_global_z,0.011,0.003,254.461,4.049,0.000
grouping1:Cond1:TPJ_L_global_z,0.000,0.003,108.676,0.142,0.887


[1] "FDR-corrected p-values"
[1] 0.019 0.011 0.011 0.634 0.887
[1] 0.646 0.042 0.646 0.015 0.000


In [18]:
print('Results with Head Motion as Covariate')
round(summary(mm_hipp_hm)$coefficients[c(14,22),],3)
round(summary(mm_ws_sochubs_hm)$coefficients[c(14,22),],3)
round(summary(mm_dmPFC_hm)$coefficients[c(14,22),],3)
round(summary(mm_sochubs_global_hm)$coefficients[c(14,22),],3)
round(summary(mm_TPJ_hm)$coefficients[c(14,22),],3)

print('FDR-corrected p-values')

print(round(p.adjust(mm_hm_3way_pvals,method="fdr"),3))
print(round(p.adjust(mm_hm_2way_pvals,method="fdr"),3))

[1] "Results with Head Motion as Covariate"


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:Hippocampus_L_global_z,0.001,0.003,139.348,0.531,0.596
grouping1:Cond1:Hippocampus_L_global_z,0.007,0.003,230.148,2.560,0.011


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:ws_sochubs_z,0.006,0.003,154.359,2.231,0.027
grouping1:Cond1:ws_sochubs_z,0.008,0.003,210.401,2.878,0.004


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:dmPFC_L_global_z,0.001,0.003,197.414,0.444,0.657
grouping1:Cond1:dmPFC_L_global_z,-0.008,0.003,191.665,-2.877,0.004


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:diffhubs_global_z,0.008,0.003,191.014,2.719,0.007
grouping1:Cond1:diffhubs_global_z,0.002,0.003,188.714,0.710,0.478


,Estimate,Std. Error,df,t value,Pr(>|t|)
grouping1:TPJ_L_global_z,0.011,0.003,254.044,3.997,0.000
grouping1:Cond1:TPJ_L_global_z,0.000,0.003,108.523,0.128,0.898


[1] "FDR-corrected p-values"
[1] 0.019 0.011 0.011 0.598 0.898
[1] 0.657 0.045 0.657 0.018 0.000


## Step 7: Correlation with Social Traits

In [19]:
traits<-read.csv(paste(wd1,"netLearn_self_report_measures_26subjs.csv",sep="/"))

# Load neural data and merge with behavioral/head motion data
diffData_gc<-create_df(wd2,"diffData_zscores","sighubs_global_conn",subjIDs,hmData,add_cov=TRUE)

diffData_ss<-create_df(wd2,"diffData_zscores","sighubs_to_sighubs",subjIDs,hmData)

#################################################
# Combine neural dataframes
#################################################


mdf<-merge(diffData_gc,diffData_ss,by = c("pID","task","CondNum","rowID","scanID"))
#longData<-merge(longData,longData_glm,by = c("pID","task","CondNum","rowID","scanID"))
mdf=merge(mdf,traits,by="pID")
mdf_soc=mdf[mdf$task=="Soc",]
mdf_ns=mdf[mdf$task=="NS",]


In [20]:
hub_var_list<-unique(c(hublist1,hublist2))
traitvars<-c("INDvsCOL","Holism","IRI_PT","Ravens_nCorr")

corr_mat<-corr.test(mdf[,c(hub_var_list,traitvars)],adjust="none")
pmat<-corr_mat$p[1:length(hub_var_list),(length(hub_var_list)+1):length(c(hub_var_list,traitvars))]
rmat<-corr_mat$r[1:length(hub_var_list),(length(hub_var_list)+1):length(c(hub_var_list,traitvars))]
fdr_pvec<-p.adjust(pmat,method="fdr")
fdr_pmat<-as.data.frame(matrix(fdr_pvec,nrow=14,ncol=4,byrow = FALSE),row.names = hub_var_list, col.names = traitvars)
names(fdr_pmat)<-traitvars
print(round(rmat,3))
print(round(fdr_pmat,3))

                     INDvsCOL Holism IRI_PT Ravens_nCorr
combhubs_global        -0.234  0.304 -0.004       -0.103
Hippocampus_L_global    0.056 -0.089 -0.399       -0.219
Hippocampus_R_global   -0.069 -0.015 -0.355       -0.070
diffhubs_global        -0.135  0.328 -0.090       -0.135
TPJ_L_global           -0.049  0.061 -0.083       -0.023
TPJ_R_global           -0.126  0.360 -0.102       -0.151
diffhubs_ns_global     -0.266  0.586  0.357       -0.252
dmPFC_L_global         -0.050  0.184  0.321       -0.247
dmPFC_R_global         -0.237  0.470  0.409       -0.279
lPFC_L_global           0.077  0.078 -0.041       -0.008
lPFC_R_global          -0.256  0.548  0.135        0.006
ws_combhubs            -0.081  0.245  0.182       -0.164
ws_nshubs              -0.265  0.042  0.307       -0.208
ws_sochubs              0.394 -0.156 -0.250        0.187
                     INDvsCOL Holism IRI_PT Ravens_nCorr
combhubs_global         0.608  0.564  0.983        0.919
Hippocampus_L_global    0.927  

## Step 8: Full Model Output for Mixed Effects Models

In [21]:
#################################################
### Loop over list of hubs and save results in list
#################################################

full_model_output<-list()
grouping_main_effect<-list()
brain_main_effect<-list()
groupxcond_int<-list()
groupxbrain_int<-list()
condxbrain_int<-list()
groupxbrainxcond_int<-list()

hublist1<-c("combhubs_global_z","Hippocampus_L_global_z","Hippocampus_R_global_z",
            "diffhubs_global_z","TPJ_L_global_z","TPJ_R_global_z",
            "diffhubs_ns_global_z","dmPFC_L_global_z","dmPFC_R_global_z","lPFC_L_global_z","lPFC_R_global_z")

#hublist2<-c("Hippocampus_L_X_combhubs_z","Hippocampus_L_X_nshubs_z","Hippocampus_L_X_sochubs_z",
#            "Hippocampus_R_X_combhubs_z","Hippocampus_R_X_nshubs_z","Hippocampus_R_X_sochubs_z",
#            "ws_combhubs_z","combhubs_X_nshubs_z","combhubs_X_sochubs_z",
#            "ws_nshubs_z","nshubs_X_sochubs_z","ws_sochubs_z")

hublist2<-c("ws_combhubs_z","ws_nshubs_z","ws_sochubs_z")

hublist<-c(hublist1,hublist2)
print("Hub analyses for node type x network type interaction")
for(x in hublist){
  mm1_cov= lmer(combData2[,"rt_log"] ~  
                  combData2[,"grouping"]*combData2[,"Cond"]*combData2[,"order"]*combData2[,"strial"]*combData2[,x] +
                  (1 + combData2[,"grouping"] + combData2[,"Cond"]+ combData2[,"strial"]| combData2[,"pID"]))
  full_model_output[[paste("all",x,sep="_")]]<-mm1_cov
  grouping_main_effect[[paste("all",x,sep="_")]]<-summary(mm1_cov)$coefficients[2,]
  brain_main_effect[[paste("all",x,sep="_")]]<-summary(mm1_cov)$coefficients[6,]
  groupxcond_int[[paste("all",x,sep="_")]]<-summary(mm1_cov)$coefficients[7,]
  groupxbrain_int[[paste("all",x,sep="_")]]<-summary(mm1_cov)$coefficients[13,]
  condxbrain_int[[paste("all",x,sep="_")]]<-summary(mm1_cov)$coefficients[14,]
  groupxbrainxcond_int[[paste("all",x,sep="_")]]<-summary(mm1_cov)$coefficients[21,]
}

[1] "Hub analyses for node type x network type interaction"


In [ ]:
#################################################
### Apply fdr-correction to p-values
#################################################

b_me_pvals_all<-list()
gxbxc_pvals_all<-list()
gxb_pvals_all<-list()

for(i in 1:length(hublist)){
  x_all<-paste("all",hublist[i],sep="_")
  b_me_pvals_all[i]<-brain_main_effect[[x_all]][5]
  gxbxc_pvals_all[i]<-groupxbrainxcond_int[[x_all]][5]
  gxb_pvals_all[i]<-groupxbrain_int[[x_all]][5]
}


# Compute fdr-corrected p-values
b_me_fdr_pvals_all<-p.adjust(b_me_pvals_all,method = 'fdr')
gxbxc_fdr_pvals_all<-p.adjust(gxbxc_pvals_all,method = 'fdr')
gxb_fdr_pvals_all<-p.adjust(gxb_pvals_all,method = 'fdr')

# Loop over hublist and add fdr-corrected p-value to results lists
for(i in 1:length(hublist)){
  x_all<-paste("all",hublist[i],sep="_")
  brain_main_effect[[x_all]]['fdr_p']<-b_me_fdr_pvals_all[i]
  groupxbrainxcond_int[[x_all]]['fdr_p']<-gxbxc_fdr_pvals_all[i]
  groupxbrain_int[[x_all]]['fdr_p']<-gxb_fdr_pvals_all[i]
}


# Print results for hublist
for(x in hublist){
  x_all<-paste("all",x,sep="_")
  pval_str1<-paste('Brain main effect: fdr p=',round(brain_main_effect[[x_all]]['fdr_p'],digits=3),'unc p=',round(brain_main_effect[[x_all]]['Pr(>|t|)'],digits=3))
  pval_str2<-paste('gxbxc 3-way interaction: fdr p=',round(groupxbrainxcond_int[[x_all]]['fdr_p'],digits=3),'unc p=',round(groupxbrainxcond_int[[x_all]]['Pr(>|t|)'],digits=3))
  pval_str3<-paste('All trials gxb interaction: fdr p=',round(groupxbrain_int[[x_all]]['fdr_p'],digits=3),'unc p=',round(groupxbrain_int[[x_all]]['Pr(>|t|)'],digits=3))
  print(x)
  print(pval_str1)
  print(pval_str2)
  print(pval_str3)
  print('')
  
}

[1] "combhubs_global_z"
[1] "Brain main effect: fdr p= 0.922 unc p= 0.772"
[1] "gxbxc 3-way interaction: fdr p= 0.846 unc p= 0.786"
[1] "All trials gxb interaction: fdr p= 0.529 unc p= 0.378"
[1] ""
[1] "Hippocampus_L_global_z"
[1] "Brain main effect: fdr p= 0.856 unc p= 0.063"
[1] "gxbxc 3-way interaction: fdr p= 0.05 unc p= 0.011"
[1] "All trials gxb interaction: fdr p= 0.698 unc p= 0.62"
[1] ""
[1] "Hippocampus_R_global_z"
[1] "Brain main effect: fdr p= 0.922 unc p= 0.535"
[1] "gxbxc 3-way interaction: fdr p= 0.233 unc p= 0.083"
[1] "All trials gxb interaction: fdr p= 0.594 unc p= 0.467"
[1] ""
[1] "diffhubs_global_z"
[1] "Brain main effect: fdr p= 0.993 unc p= 0.993"
[1] "gxbxc 3-way interaction: fdr p= 0.679 unc p= 0.487"
[1] "All trials gxb interaction: fdr p= 0.049 unc p= 0.007"
[1] ""
[1] "TPJ_L_global_z"
[1] "Brain main effect: fdr p= 0.922 unc p= 0.742"
[1] "gxbxc 3-way interaction: fdr p= 0.88 unc p= 0.88"
[1] "All trials gxb interaction: fdr p= 0.001 unc p= 0"
[1] ""
[1] "T

In [ ]:
b_me_pvals_all

[[1]]
[1] 0.7718625

[[2]]
[1] 0.06325971

[[3]]
[1] 0.5348237

[[4]]
[1] 0.9929538

[[5]]
[1] 0.7417804

[[6]]
[1] 0.7271398

[[7]]
[1] 0.1339551

[[8]]
[1] 0.8982231

[[9]]
[1] 0.3463588

[[10]]
[1] 0.5122267

[[11]]
[1] 0.7904438

[[12]]
[1] 0.6975401

[[13]]
[1] 0.1834499

[[14]]
[1] 0.546298

### Print full model output for each model

In [ ]:
for(x in hublist){
    output<-summary(full_model_output[[paste('all',x,sep='_')]])$coefficient[c(6),]
    #row.names(output)<-gsub(" ","",row.names(output))
    #row.names(output)<-gsub("combData2_ss","",row.names(output))
    #row.names(output)<-gsub("combData2_gc","",row.names(output))
    
    print(paste(x,'_Main_Effect',sep=""))
    print(round(output,digits=3))
    print('')
}

[1] "combhubs_global_z_Main_Effect"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.003      0.010     23.914      0.293      0.772 
[1] ""
[1] "Hippocampus_L_global_z_Main_Effect"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.020      0.010     25.168      1.943      0.063 
[1] ""
[1] "Hippocampus_R_global_z_Main_Effect"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.006      0.009     24.701      0.629      0.535 
[1] ""
[1] "diffhubs_global_z_Main_Effect"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.000      0.011     25.655      0.009      0.993 
[1] ""
[1] "TPJ_L_global_z_Main_Effect"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.004      0.011     24.826      0.333      0.742 
[1] ""
[1] "TPJ_R_global_z_Main_Effect"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.004      0.011     25.680      0.353      0.727 
[1] ""
[1] "diffhubs_ns_global_z_Main_Effect"
  Estimate Std. Error      

In [ ]:
for(x in hublist){
    output<-summary(full_model_output[[paste('all',x,sep='_')]])$coefficient[c(13),]
    #row.names(output)<-gsub(" ","",row.names(output))
    #row.names(output)<-gsub("combData2_ss","",row.names(output))
    #row.names(output)<-gsub("combData2_gc","",row.names(output))
    
    print(paste('NodeTypex',x,sep=""))
    print(round(output,digits=3))
    print('')
}

[1] "NodeTypexcombhubs_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.002      0.003    293.809      0.883      0.378 
[1] ""
[1] "NodeTypexHippocampus_L_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.001      0.003    139.794      0.497      0.620 
[1] ""
[1] "NodeTypexHippocampus_R_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.002      0.003    334.956      0.728      0.467 
[1] ""
[1] "NodeTypexdiffhubs_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.008      0.003    190.896      2.730      0.007 
[1] ""
[1] "NodeTypexTPJ_L_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.011      0.003    253.785      4.018      0.000 
[1] ""
[1] "NodeTypexTPJ_R_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.006      0.003    178.551      2.069      0.040 
[1] ""
[1] "NodeTypexdiffhubs_ns_global_z"
  Estimate Std. Error         df    t value   Pr

In [ ]:
for(x in hublist){
    output<-summary(full_model_output[[paste('all',x,sep='_')]])$coefficient[c(21),]
    #row.names(output)<-gsub(" ","",row.names(output))
    #row.names(output)<-gsub("combData2_ss","",row.names(output))
    #row.names(output)<-gsub("combData2_gc","",row.names(output))
    
    print(paste('NodeTypexNetworkTypex',x,sep=""))
    print(round(output,digits=3))
    print('')
}

[1] "NodeTypexNetworkTypexcombhubs_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.001      0.003    159.363      0.272      0.786 
[1] ""
[1] "NodeTypexNetworkTypexHippocampus_L_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.007      0.003    231.161      2.571      0.011 
[1] ""
[1] "NodeTypexNetworkTypexHippocampus_R_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.005      0.003    103.790      1.748      0.083 
[1] ""
[1] "NodeTypexNetworkTypexdiffhubs_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.002      0.003    188.561      0.697      0.487 
[1] ""
[1] "NodeTypexNetworkTypexTPJ_L_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.000      0.003    108.452      0.152      0.880 
[1] ""
[1] "NodeTypexNetworkTypexTPJ_R_global_z"
  Estimate Std. Error         df    t value   Pr(>|t|) 
     0.002      0.003    185.543      0.917      0.360 
[1] ""
[1] "NodeTyp

Steven Tompson | 2019